In [151]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math

# Modelo general

In [152]:
model = gp.Model("Generación de horarios de colegio")
model.setParam('OutputFlag', 1)

M = 10000
C_ = ['A','B','C','D'] # Cursos
I_ = [1,2,3,4] # Niveles
tope_modulos = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
J_ = {
    'Lunes': [1, 2, 3, 4, 5, 6], 
    'Martes': [1, 2, 3, 4, 5, 6, 7, 8], 
    'Miércoles': [1, 2, 3, 4, 5, 6], 
    'Jueves': [1, 2, 3, 4, 5, 6, 7, 8], 
    'Viernes': [1, 2, 3, 4, 5, 6]
} # Módulos
combinaciones = [
    (1,2),(3,4),(5,6),(7,8)
]
K_ = [
    'Lenguaje', 'Matemáticas', 'Educación_Física', 'Historia', 'Ciencias_Naturales', 'Inglés', 'Artes_Visuales',
    'Religión', 'Tecnología', 'Música', 'Orientación_Pastoral'
]
D_ = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']
P_ = [
    'Alexis_Sanchez', 'Arturo_Vidal', 'Claudio_Bravo', 
    'Gary_Medel', 
    'Harry_Potter', 'Ron_Weasley', 'Hermione_Granger', 'Draco_Malfoy',
    'Harry_Potter2', 'Ron_Weasley2', 'Hermione_Granger2', 'Draco_Malfoy2',
    'Harry_Potter3', 'Ron_Weasley3', 'Hermione_Granger3', 'Draco_Malfoy3',
    'Harry_Potter4', 'Ron_Weasley4', 'Hermione_Granger4', 'Draco_Malfoy4',
    'Lionel_Messi', 'Kylian_Mbappé', 'Luka_Modric', 'Achraf_Hakimi',
    'Lionel_Messi2', 'Kylian_Mbappé2', 'Luka_Modric2', 'Achraf_Hakimi2',
    'Lionel_Messi3', 'Kylian_Mbappé3', 'Luka_Modric3', 'Achraf_Hakimi3',
]

clases_por_profesor = {
    'Alexis_Sanchez': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
    ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)],
    'Arturo_Vidal': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
    ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)],
    'Claudio_Bravo': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
    ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)],
    'Gary_Medel': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1)],

    'Harry_Potter': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
    ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)],    
    'Ron_Weasley': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
    ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)],
    'Hermione_Granger': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
    ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)],
    'Draco_Malfoy': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
    ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)],

    'Harry_Potter2': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
    ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)],    
    'Ron_Weasley2': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
    ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)],
    'Hermione_Granger2': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
    ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)],
    'Draco_Malfoy2': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
    ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)],

    'Harry_Potter3': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
    ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)],    
    'Ron_Weasley3': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
    ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)],
    'Hermione_Granger3': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
    ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)],
    'Draco_Malfoy3': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
    ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)],

    'Harry_Potter4': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
    ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)],    
    'Ron_Weasley4': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
    ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)],
    'Hermione_Granger4': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
    ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)],
    'Draco_Malfoy4': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
    ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)],

    'Lionel_Messi': [('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
    ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)],    
    'Kylian_Mbappé': [('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
    ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)],
    'Luka_Modric': [('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
    ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)],
    'Achraf_Hakimi': [('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
    ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)],

    'Lionel_Messi2': [
        # ('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
        ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)
    ],    
    'Kylian_Mbappé2': [
        # ('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
        ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)
    ],
    'Luka_Modric2': [
        # ('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
        ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)
    ],
    'Achraf_Hakimi2': [
        # ('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
        ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)
    ],

    'Lionel_Messi3': [
        # ('Lenguaje', 1), ('Artes_Visuales', 1), ('Inglés', 1), ('Lenguaje', 2), ('Artes_Visuales', 2), ('Inglés', 2), 
        ('Lenguaje', 3), ('Artes_Visuales', 3), ('Inglés', 3), ('Lenguaje', 4), ('Artes_Visuales', 4), ('Inglés', 4)
    ],    
    'Kylian_Mbappé3': [
        # ('Religión', 2), ('Música', 2), ('Orientación_Pastoral', 2), ('Religión', 1), ('Música', 1), ('Orientación_Pastoral', 1), 
        ('Religión', 3), ('Música', 3), ('Orientación_Pastoral', 3), ('Religión', 4), ('Música', 4), ('Orientación_Pastoral', 4)
    ],
    'Luka_Modric3': [
        # ('Matemáticas', 1), ('Historia', 1), ('Ciencias_Naturales', 1), ('Matemáticas', 2), ('Historia', 2), ('Ciencias_Naturales', 2),
        ('Matemáticas', 3), ('Historia', 3), ('Ciencias_Naturales', 3), ('Matemáticas', 4), ('Historia', 4), ('Ciencias_Naturales', 4)
    ],
    'Achraf_Hakimi3': [
        # ('Tecnología', 2), ('Educación_Física', 2), ('Tecnología', 1), ('Educación_Física', 1), 
        ('Tecnología', 3), ('Educación_Física', 3), ('Tecnología', 4), ('Educación_Física', 4)
    ],
}

clases_por_nivel = {
    ('Lenguaje', 1): 8,
    ('Matemáticas', 1): 6,
    ('Educación_Física', 1): 4,
    ('Historia', 1): 3,
    ('Ciencias_Naturales', 1): 3,
    ('Inglés', 1): 2,
    ('Artes_Visuales', 1): 2, 
    ('Religión', 1): 2,
    ('Tecnología', 1): 1,
    ('Orientación_Pastoral', 1): 1,
    ('Música', 1): 2, 
    
    ('Lenguaje', 2): 8,
    ('Matemáticas', 2): 6,
    ('Educación_Física', 2): 4,
    ('Historia', 2): 3,
    ('Ciencias_Naturales', 2): 3,
    ('Inglés', 2): 2,
    ('Artes_Visuales', 2): 2, 
    ('Religión', 2): 2,
    ('Tecnología', 2): 1,
    ('Orientación_Pastoral', 2): 1,
    ('Música', 2): 2,

    ('Lenguaje', 3): 8,
    ('Matemáticas', 3): 6,
    ('Educación_Física', 3): 4,
    ('Historia', 3): 3,
    ('Ciencias_Naturales', 3): 3,
    ('Inglés', 3): 2,
    ('Artes_Visuales', 3): 2, 
    ('Religión', 3): 2,
    ('Tecnología', 3): 1,
    ('Orientación_Pastoral', 3): 1,
    ('Música', 3): 2,

    ('Lenguaje', 4): 8,
    ('Matemáticas', 4): 6,
    ('Educación_Física', 4): 4,
    ('Historia', 4): 3,
    ('Ciencias_Naturales', 4): 3,
    ('Inglés', 4): 2,
    ('Artes_Visuales', 4): 2, 
    ('Religión', 4): 2,
    ('Tecnología', 4): 1,
    ('Orientación_Pastoral', 4): 1,
    ('Música', 4): 2
}


x = model.addVars(K_, D_, tope_modulos, I_, C_, P_, vtype=GRB.BINARY, name='x')
y = model.addVars(K_, I_, C_, P_, vtype=GRB.BINARY, name='y')
s = model.addVars(K_, D_, I_, C_, vtype=GRB.BINARY, name='s')

# Restricciones del modelo

##### Una clase por módulo y solo en los que se permite

In [153]:
model.addConstrs((sum(x[k,d,j,i,c,p] for k in K_ for p in P_) == 1 for d in D_ for j in J_[d] for i in I_ for c in C_),name="RX1");

In [154]:
model.addConstrs((sum(x[k,d,j,i,c,p] for k in K_ for p in P_) == 0 for d in D_ for j in list(set(tope_modulos) - set(J_[d])) for i in I_ for c in C_),name="RX2");

##### Cumplir con la cantidad de módulos de clases de todos los ramos

In [155]:
model.addConstrs((sum(x[k,d,j,i,c,p] for d in D_ for j in J_[d] for p in P_) == clases_por_nivel[k, i] for k in K_ for i in I_ for c in C_),name="RX3");

##### Máximo de dos clases de la misma materia en un día y si es que hay, que sean seguidas

In [156]:
model.addConstrs((sum(x[k,d,j,i,c,p] for j in J_[d] for p in P_) <= 2 for k in K_ for d in D_ for i in I_ for c in C_),name="RX4");

In [157]:
model.addConstrs((x[k,d,j,i,c,p] + x[k,d,z,i,c,p] <= 1 for k in K_ for d in D_ for i in I_ for c in C_ for j,z in list(set([(a,b) for a in J_[d] for b in J_[d] if a < b]) - set(combinaciones)) for p in P_),name="RX5");

##### Restricciones profesores

Los profesores no hacen los ramos que no dictan

In [158]:
model.addConstrs((x[k,d,j,i,c,p] == 0 for p in P_ for d in D_ for j in J_[d] for c in C_ for k, i in list(set([(a,b) for a in K_ for b in I_]) - set(clases_por_profesor[p]))),name="RX6");

Los profesores solo hacen una clase en un módulo a la vez

In [159]:
model.addConstrs((sum(x[k,d,j,i,c,p] for k in K_ for i in I_ for c in C_) <= 1 for p in P_ for d in D_ for j in J_[d]),name="RX7");

El mismo profesor le hace todas las clases de un ramo a un mismo curso

In [160]:
model.addConstrs((M * y[k,i,c,p] >= sum(x[k,d,j,i,c,p] for d in D_ for j in J_[d]) for k in K_ for i in I_ for c in C_ for p in P_),name="RY1");

In [161]:
model.addConstrs((sum(y[k,i,c,p] for p in P_) <= 1 for k in K_ for i in I_ for c in C_),name="RY2");

##### Funcionamiento variable s

In [162]:
model.addConstrs((M * s[k,d,i,c] >= sum(x[k,d,j,i,c,p] for j in J_[d] for p in P_) for k in K_ for d in D_ for i in I_ for c in C_),name="RS1");

In [163]:
model.addConstrs((sum(s[k,d,i,c] for d in D_) <= math.ceil(clases_por_nivel[k,i] / 2) for k in K_ for d in D_ for i in I_ for c in C_),name="RS2");

##### Correr modelo

In [164]:
obj = sum(
    x[k,d,j,i,c,p] for k in K_ for d in D_ for j in J_[d] for i in I_ for c in C_ for p in P_
)

model.setObjective(obj, GRB.MINIMIZE)
# model.setObjective(0, GRB.MINIMIZE)
model.update()
# model.optimize()
model.write('model.lp')

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

# model.setParam('TimeLimit', 60)
model.setParam('SolutionLimit', 1)

model.optimize()
model.write('out.sol')

Set parameter SolutionLimit to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 633744 rows, 288112 columns and 2351904 nonzeros
Model fingerprint: 0x46149fc6
Variable types: 0 continuous, 288112 integer (288112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 600096 rows and 245216 columns
Presolve time: 4.24s
Presolved: 33648 rows, 42896 columns, 346576 nonzeros
Variable types: 0 continuous, 42896 integer (42896 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4400000e+02   2.174830e+01   9.545000e+08      5s
     732    5.4400000e+02   0.000000e+00   0.000000e+00      5s
     732    5.4400000

In [165]:
lista_final = []
with open('out.sol', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line[-2] == '1':
            line = line.strip('\n')
            lista_final.append(line)
# print(lista_final)
diccionario_dias = {
    'Lunes': 1,
    'Martes': 2,
    'Miércoles': 3,
    'Jueves': 4,
    'Viernes': 5
}

vuelta = {
    1: 'Lunes',
    2: 'Martes',
    3: 'Miércoles',
    4: 'Jueves',
    5: 'Viernes'
}

profesores = []
profesores = []
for elem in lista_final:
    if elem[0] == "x":
        elem = elem[2:-3]
        elem = elem.split(',')
        profesores.append(elem)
for i in range(len(profesores)):
    profesores[i][1] = diccionario_dias[profesores[i][1]]
profesores = sorted(profesores, key= lambda x: x[2])
profesores = sorted(profesores, key= lambda x: x[1])
profesores = sorted(profesores, key= lambda x: x[4])

for i in range(len(profesores)):
    profesores[i][1] = vuelta[profesores[i][1]]




# print(profesores)
# print(profesores)
# for elem in profesores:
#     if elem[3] == '2' and elem[4] == 'A':
#         print(elem)
# for elem in profesores:
#     if elem[0] == 'Lenguaje':
#         print(elem)

In [167]:
from prettytable import PrettyTable

def generar_horario(curso):
    nivel = curso[:-1]
    paralelo = curso[-1]

    clases_curso = []
    dias = []
    ultimo_dia = None
    horario = PrettyTable(dias)
    lista_auxiliar = []
    lista_dia = None
    for elem in profesores:    
        if elem[3] == nivel and elem[4] == paralelo:
            clases_curso.append([elem[0], elem[1], elem[2], elem[5]])
            if elem[1] not in dias:
                dias.append(elem[1])
            if elem[1] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[1]
                lista_dia.append(ultimo_dia)
                elem[0] = elem[0].replace('_', ' ')
                elem[5] = elem[5].replace('_', ' ')
                lista_dia.append(f'{elem[0]} \n {elem[5]}')
            else:
                elem[0] = elem[0].replace('_', ' ')
                elem[5] = elem[5].replace('_', ' ')
                lista_dia.append(f'{elem[0]} \n {elem[5]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]

    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])

    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '4A'
generar_horario(curso)

Horario del curso 4A: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Historia Hermione Granger,Lenguaje Harry Potter,Matemáticas Hermione Granger,Historia Hermione Granger,Lenguaje Harry Potter
2,Orientación Pastoral Kylian Mbappé3,Lenguaje Harry Potter,Matemáticas Hermione Granger,Historia Hermione Granger,Lenguaje Harry Potter
3,Educación Física Draco Malfoy,Ciencias Naturales Hermione Granger3,Lenguaje Harry Potter,Ciencias Naturales Hermione Granger3,Música Arturo Vidal
4,Educación Física Draco Malfoy,Ciencias Naturales Hermione Granger3,Lenguaje Harry Potter,Tecnología Draco Malfoy,Música Arturo Vidal
5,Lenguaje Harry Potter,Matemáticas Hermione Granger,Educación Física Draco Malfoy,Inglés Lionel Messi3,Matemáticas Hermione Granger
6,Lenguaje Harry Potter,Matemáticas Hermione Granger,Educación Física Draco Malfoy,Inglés Lionel Messi3,Matemáticas Hermione Granger
7,,Artes Visuales Lionel Messi,,Religión Arturo Vidal,
8,,Artes Visuales Lionel Messi,,Religión Arturo Vidal,
